In [1]:
import pickle
import pandas as pd
import numpy as np
import streamlit as st
from model import ModelClinicValue
import corporate_wellness_app
import school_outreach_app
from model_forecasting import ModelForecastPerformance
from cashflow_plot import ModelCashflow
from streamlit_extras.stateful_button import button
import structured_saving as structured_saving_app
import os
    


In [2]:
    
clinic_data_set = pickle.load(open(r'dummy_clinic_model\pkl_files\dataset_1\clinic_value_set.pkl', 'rb'))
clinic_historical_demand = pickle.load(open(f'dummy_clinic_model/pkl_files/dataset_1/clinic_metrics.pkl', 'rb'))
    
dataset = 1
    
selected_clinic_set = list(clinic_data_set.keys())

In [3]:
# SAVE INTO PKL

file_name = 'clinic_projected_cashflow_set.pkl'


# Initialize or load existing dictionary
if os.path.exists(file_name):
    # Load existing data
    with open(file_name, 'rb') as f:
        clinic_projected_cashflow_set = pickle.load(f)
else:
    # Start with an empty dictionary
    clinic_projected_cashflow_set = {}

In [4]:
# Initialize or load existing dictionary
if os.path.exists('forecast_dictionary.pkl'):
    # Load existing data
    with open('forecast_dictionary.pkl', 'rb') as f:
        forecast_dictionary = pickle.load(f)
else:
    # Start with an empty dictionary
    forecast_dictionary = {}   

In [ ]:

        
        
        
for selected_clinic in selected_clinic_set:
            
    clinic_data = clinic_data_set[selected_clinic]

    clinic_value = clinic_data['Clinic Valuation Adjusted']
    item_code_clinic_historical = clinic_historical_demand[selected_clinic]['updated_clinic_item_code']

    model = ModelForecastPerformance(clinic_data)

    
    percentage_borrow = 50
    upfront_payment = 20
                    

    interest_rate = 12
    loan_term = 5
    buying_year = 2025
             
    acquiring_funding = "Borrow"       
                    
    borrowed = (clinic_value * (percentage_borrow/100))
    principal = borrowed - ((upfront_payment/100) * borrowed)
    amortization_df, monthly_payment, total_principal, total_interest = model.loan_amortization_schedule(principal, interest_rate, loan_term, start_year=buying_year)
                
    sliced_amortization_df = amortization_df[['Period', 'Monthly Payment']]
    sliced_amortization_df = sliced_amortization_df.rename(columns={'Monthly Payment':"Expense"})
    sliced_amortization_df['Revenue'] = 0
                

    period_forecast = 12

    start_year = 2025
        

    number_of_months = model.total_days_from_start(period_forecast, start_year=start_year) 
    model_cashflow = ModelCashflow()
    forecast_df = model.forecast_revenue_expenses(period_forecast, start_month=1)
    indirect_expense = model.forecast_indirect_cost(period_forecast, start_year=start_year)
            
            

    treatment_details = pd.read_csv("treatment_with_details.csv")
    item_code_df = pd.read_csv("dummy_clinic_model\cleaned_item_code.csv")
            
    
    forecast_df_with_treatments = model.generate_forecast_item_code_by_profit(item_code_df, forecast_df, item_code_clinic_historical, patient_pool=clinic_data['Patient Pool'], start_year=start_year)
            

            
    forecast_df = forecast_df_with_treatments[['Period', 'Revenue', 'Expense']].groupby('Period').sum().reset_index()


    sliced_amortization_df = sliced_amortization_df[:period_forecast] if acquiring_funding == "Borrow" else None
        
    equipment_df = model.equipment_to_cashflow(clinic_data['Equipment Life'], period_forecast, start_year=start_year)


    fitout_df = model.fitout_to_cashflow(clinic_data['Fitout Value'], clinic_data['Last Fitout Year'] ,period_forecast, start_year=start_year)

    model_cashflow.add_company_data("Gross Profit", forecast_df)
    model_cashflow.add_company_data("Debt Repayment", sliced_amortization_df) if acquiring_funding == "Borrow" else None
    model_cashflow.add_company_data("Indirect Expense", indirect_expense)
    model_cashflow.add_company_data("Equipment Procurement", equipment_df)
    model_cashflow.add_company_data("Fit Out", fitout_df)
    
    without_improvement_df = model_cashflow.merge_dataframes_with_category()
    
    without_improvement_df_profit = model_cashflow.groupby_dataframes_month_year(without_improvement_df)['Profit'].values
    
    # set without_improvement_df_profit to list
    without_improvement_df_profit = without_improvement_df_profit.tolist()
    

            
    clinic_value = clinic_data['Clinic Valuation Adjusted']
    clinic_ebit_multiple = clinic_data['EBIT Multiple']


    current_ebit = clinic_data['EBIT']
    current_ebit_ratio = clinic_data['EBIT Ratio']
    current_growth = clinic_data['Net Sales Growth']
    current_net_sales_relative_variation = clinic_data['Relative Variation of Net Sales']
            



        
    MA_forecast_df = model.forecast_revenue_expenses(3) # instead of using actual data, this method forecast three months as basis of most recent months of historical data 
                
    MA_forecast_df = model.forecast_revenue_expenses_MA(MA_forecast_df, period_forecast, 3, start_month=1)
                
                
                
                # MA_forecast_df_with_treatments = model.generate_forecast_treatment_df_by_profit(treatment_details, MA_forecast_df)
    MA_forecast_df_with_treatments = model.generate_forecast_item_code_by_profit(item_code_df, MA_forecast_df, item_code_clinic_historical, start_year=start_year)
    MA_forecast_df = MA_forecast_df_with_treatments[['Period', 'Revenue', 'Expense']].groupby('Period').sum().reset_index()

    MA_indirect_cost = model.forecast_indirect_cost(period_forecast, start_year=start_year)

                
                # model_cashflow.remove_all_companies()
                # model_cashflow.add_company_data("Gross Profit", MA_forecast_df)
                # model_cashflow.add_company_data("Debt Repayment", sliced_amortization_df) if acquiring_funding == "Borrow" else None
                # model_cashflow.add_company_data("Indirect Expense", MA_indirect_cost)
                # model_cashflow.add_company_data("Equipment Procurement", equipment_df)
                # model_cashflow.add_company_data("Fit Out", fitout_df)
                
                # MA_forecast_linechart_daily = model_cashflow.cashflow_plot(number_of_months, start_date=generate_datetime(start_year=start_year))
                # MA_forecast_linechart_weekly = model_cashflow.cashflow_plot(number_of_months, granularity='weekly', start_date=generate_datetime(start_year=start_year))
                # MA_forecast_linechart_monthly = model_cashflow.cashflow_plot(number_of_months, granularity='monthly', start_date=generate_datetime(start_year=start_year))
                
            
            # current_ebit_after_12_months = current_ebit * (1+clinic_data['Net Sales Growth'])


            # potential_ebit_after_12_months = current_ebit_after_12_months
            
            
            
    ID = selected_clinic
            
    
    clinic_projected_cashflow_set[ID] = {
        "Approach 1": 
            {"Gross Profit": forecast_df,
            "Debt Repayment": sliced_amortization_df if acquiring_funding == "Borrow" else None,
            "Indirect Expense": indirect_expense,
            "Equipment Procurement": equipment_df,
            "Fit Out": fitout_df},
        "Approach 2": 
            {"Gross Profit": MA_forecast_df,
            "Debt Repayment": sliced_amortization_df if acquiring_funding == "Borrow" else None,
            "Indirect Expense": MA_indirect_cost,
            "Equipment Procurement": equipment_df,
            "Fit Out": fitout_df}
    }
    
    
    forecast_dictionary[ID] = {
    'without_improvement_df_profit': without_improvement_df_profit,
    'forecast_df_with_treatments': forecast_df_with_treatments,
    'dataset': dataset,
    'selected_clinic': selected_clinic,
    'period_forecast': period_forecast,
    'start_year': start_year,
    'acquiring_funding': acquiring_funding,
    'sliced_amortization_df': sliced_amortization_df,
    'indirect_expense': indirect_expense,
    'equipment_df': equipment_df,
    'fitout_df': fitout_df,
    'forecast_df': forecast_df,
    'model_cashflow': model_cashflow,
    'number_of_months': number_of_months,
    'without_improvement_df': without_improvement_df


    }
            

    



        

        # st.markdown("### Strategy to implement")

        # col1, col2 = st.columns(2)

        # with col1:
        #     corporate_wellness = st.checkbox("Corporate Wellness Program", value=False)
        #     structured_saving = st.checkbox("Structured Saving Plan", value=False)
        #     fair_credit = st.checkbox("Fair Credit Program", value=False)
            
        # with col2:
        #     with st.popover("details"):
        #         corporate_wellness_app.app()
                
        #     with st.popover("details"):
        #         structured_saving_app.app()
                
        # # col1, col2 = st.columns(2)
        
        # # with col1:
        # #     school_outreach = st.checkbox("School Outreach Program", value=False)
            
        # # with col2:
        # #     with st.popover("details"):
        # #         school_outreach_app.app()
            
        


        # if button("Calculate New Cash Flow", key="calculate-strategy"):
            
        #     corporate_wellness_df = None
        #     structured_saving_df = None
        #     fair_credit_df = None
            
        #     if corporate_wellness == True:
        #         corporate_wellness_df = pd.read_csv("corporate_cashflow_AUD.csv")[:period_forecast]
        #         corporate_wellness_df['Period'] = corporate_wellness_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Corporate Wellness"] = corporate_wellness_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Corporate Wellness"] = corporate_wellness_df
                
        #     if structured_saving == True:
        #         structured_saving_df = pd.read_csv("structured_saving_cashflow.csv")[:period_forecast]
        #         structured_saving_df['Period'] = structured_saving_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Structured Saving"] = structured_saving_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Structured Saving"] = structured_saving_df
            
        #     if fair_credit == True:
        #         fair_credit_df = pd.read_csv("fair_credit_cashflow.csv")[:period_forecast]
        #         fair_credit_df['Period'] = fair_credit_df['Period'].apply(lambda period: model.generate_date_from_month(int(period), method='first_day', start_year=start_year))
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 1"]["Fair Credit"] = fair_credit_df
        #         clinic_projected_cashflow_set[selected_clinic]["Approach 2"]["Fair Credit"] = fair_credit_df

        #     model_cashflow.add_company_data("Corporate Wellness", corporate_wellness_df) if corporate_wellness_df is not None else None
        #     model_cashflow.add_company_data("Structured Saving", structured_saving_df) if structured_saving_df is not None else None
        #     model_cashflow.add_company_data("Fair Credit", fair_credit_df) if fair_credit_df is not None else None
        #     st.plotly_chart(model_cashflow.cashflow_plot(number_of_months, granularity='monthly', start_date=generate_datetime(start_year=start_year)))
            
            
        # st.write(clinic_projected_cashflow_set)

In [6]:
len(clinic_projected_cashflow_set)

50

In [7]:
len(forecast_dictionary)

17

In [6]:
# save into pkl
with open(file_name, 'wb') as f:
    pickle.dump(clinic_projected_cashflow_set, f)

In [ ]:


with open("forecast_dictionary.pkl", 'wb') as f:
    pickle.dump(forecast_dictionary, f)

### Transforming into JSON

In [26]:
import pickle
import pandas as pd 
import numpy as np

# load pickle

file_name = 'clinic_projected_cashflow_set.pkl'

with open(file_name, 'rb') as f:
    clinic_projected_cashflow_set = pickle.load(f)
    


In [27]:
len(clinic_projected_cashflow_set)

# save to json file in local


50

In [28]:
result = {}
for level1_key, level1_value in clinic_projected_cashflow_set.items():
    if 'Approach 1' in level1_value:
        # Include only Level 3 under Level 2-A
        result[level1_key] = level1_value['Approach 1']

result

{'N21701': {'Gross Profit':         Period  Revenue     Expense
  0   2025-01-01    675.0  128.043333
  1   2025-01-02    735.0  434.690000
  2   2025-01-03   1218.0  351.550000
  3   2025-01-04    962.0  491.516667
  4   2025-01-05   1658.0  216.470000
  ..         ...      ...         ...
  694 2026-12-27   1541.0  202.516667
  695 2026-12-28   1598.0  487.736667
  696 2026-12-29   1919.0  405.930000
  697 2026-12-30   1178.0  219.100000
  698 2026-12-31   1795.0  437.870000
  
  [699 rows x 3 columns],
  'Debt Repayment':        Period  Expense  Revenue
  0  2025-01-01  5875.42        0
  1  2025-02-01  5875.42        0
  2  2025-03-01  5875.42        0
  3  2025-04-01  5875.42        0
  4  2025-05-01  5875.42        0
  5  2025-06-01  5875.42        0
  6  2025-07-01  5875.42        0
  7  2025-08-01  5875.42        0
  8  2025-09-01  5875.42        0
  9  2025-10-01  5875.42        0
  10 2025-11-01  5875.42        0
  11 2025-12-01  5875.42        0
  12 2026-01-01  5875.42     

In [29]:
import pandas as pd

def merge_dataframes_with_category(collection_dfs):
    """
    Merges multiple DataFrames from a dictionary into a single DataFrame 
    with an additional column indicating the category.

    Parameters:
        collection_dfs (dict): A dictionary where keys are category names 
                               and values are DataFrames with identical columns 
                               'Period', 'Revenue', and 'Expense'.

    Returns:
        pd.DataFrame: A single DataFrame with an additional 'Category' column.
    """
    # List to store dataframes with the Category column added
    merged_dfs = []
    
    for category, df in collection_dfs.items():
        # Add the 'Category' column to the current dataframe
        df_with_category = df.copy()
        df_with_category['Category'] = category
        # Append the modified dataframe to the list
        merged_dfs.append(df_with_category)
    
    # Concatenate all dataframes in the list
    merged_df = pd.concat(merged_dfs, ignore_index=True)
    
    
    merged_df = merged_df[['Period', 'Revenue', 'Expense', 'Category']]
    
    return merged_df


In [30]:
clinic_projected_cashflow_set

{'N21701': {'Approach 1': {'Gross Profit':         Period  Revenue     Expense
   0   2025-01-01    675.0  128.043333
   1   2025-01-02    735.0  434.690000
   2   2025-01-03   1218.0  351.550000
   3   2025-01-04    962.0  491.516667
   4   2025-01-05   1658.0  216.470000
   ..         ...      ...         ...
   694 2026-12-27   1541.0  202.516667
   695 2026-12-28   1598.0  487.736667
   696 2026-12-29   1919.0  405.930000
   697 2026-12-30   1178.0  219.100000
   698 2026-12-31   1795.0  437.870000
   
   [699 rows x 3 columns],
   'Debt Repayment':        Period  Expense  Revenue
   0  2025-01-01  5875.42        0
   1  2025-02-01  5875.42        0
   2  2025-03-01  5875.42        0
   3  2025-04-01  5875.42        0
   4  2025-05-01  5875.42        0
   5  2025-06-01  5875.42        0
   6  2025-07-01  5875.42        0
   7  2025-08-01  5875.42        0
   8  2025-09-01  5875.42        0
   9  2025-10-01  5875.42        0
   10 2025-11-01  5875.42        0
   11 2025-12-01  5875.

In [31]:
merge_dataframes_with_category(clinic_projected_cashflow_set['N21701']['Approach 1'])

,Period,Revenue,Expense,Category
0,2025-01-01,675.0,128.043333,Gross Profit
1,2025-01-02,735.0,434.690000,Gross Profit
2,2025-01-03,1218.0,351.550000,Gross Profit
3,2025-01-04,962.0,491.516667,Gross Profit
4,2025-01-05,1658.0,216.470000,Gross Profit
...,...,...,...,...
790,2026-08-01,0.0,0.000000,Fit Out
791,2026-09-01,0.0,0.000000,Fit Out
792,2026-10-01,0.0,0.000000,Fit Out
793,2026-11-01,0.0,0.000000,Fit Out


In [35]:
final_dict = {}

for key, value in result.items():

    dataframe = merge_dataframes_with_category(value).drop(columns=['Category Equipment']) if 'Category Equipment' in merge_dataframes_with_category(value).columns else merge_dataframes_with_category(value)
    dataframe['Period'] = pd.to_datetime(dataframe['Period'])
    dataframe['Period'] = dataframe['Period'].dt.strftime('%Y-%m-%d')
    final_dict[key] = dataframe.to_dict(orient='records')
    
    
final_dict


{'N21701': [{'Period': '2025-01-01',
   'Revenue': 675.0,
   'Expense': 128.04333333333332,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-02',
   'Revenue': 735.0,
   'Expense': 434.69,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-03',
   'Revenue': 1218.0,
   'Expense': 351.55,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-04',
   'Revenue': 962.0,
   'Expense': 491.51666666666665,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-05',
   'Revenue': 1658.0,
   'Expense': 216.47,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-06',
   'Revenue': 236.0,
   'Expense': 183.23000000000002,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-07',
   'Revenue': 389.0,
   'Expense': 83.38,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-08',
   'Revenue': 3828.0,
   'Expense': 745.4333333333334,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-09',
   'Revenue': 778.0,
   'Expense': 413.18,
   'Category': 'Gross Profit'},
  {'Period': '2025-01-1

In [37]:
import json

# save final dict to json file 
with open('data_klinik.json', 'w') as f:
    json.dump(final_dict, f)
